# 모듈 불러오기

In [1]:
import os
import re
import pandas as pd
import openpyxl
import kss
from pykospacing import spacing
from nltk.tokenize import sent_tokenize
from math import log

/home/kpkim/.pyenv/versions/3.7.7/lib/python3.7/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


# 지식인 크롤링 파일 불러오기 (전문가+랭커)

In [2]:
with open('../../Data/Naver_kin_QA(estate)/qa_expert_n_ranker.txt', 'r') as f:
    qa_lines = f.readlines()

In [3]:
len(qa_lines)

158024

# 데이터 형태 확인

## 질문 제목

In [4]:
for i in range(0, 10, 5):
    print(qa_lines[i])

권리금에 대한 질문요

친구 전기세 가스비 청구



## 질문 내용

In [5]:
for i in range(1, 10, 5):
    print(qa_lines[i])

안녕하세요 권리금에 대해 질문이 있습니다.울산에서 조그만한 맥주집을 운영합니다.계약당시 계약서에 "단 권리금은 없다"라는 문구를 넣어서 계약을 했습니다.그러면 저는 계약 종료후 나갈때 새로운 임대인에게 권리금을 청구하지 못하는건가요?답변 부탁드립니다.꾸벅~

제가 사정이있어서 두달정도 월세집에 친구 혼자 냅두고 저는 고향을 내려갔어요 한달정도 살고 친구는 짐 정리를 하고 나갔는데 친구가 불이랑 보일러를 안끄고 가서 그게 한달동안 틀어져있다면 나온 전기세랑 가스비를 법적으로 받을 방법이있나요 ?



## 답변 내용

In [6]:
for i in range(2, 10, 5):
    print(qa_lines[i])

안녕하세요. 계약, 부동산, 임대차 등 재산분쟁을 주요 업무로 하는서울지방변호사회-네이버 지식iN 상담 변호사 이동영 입니다.​상가임대차에 관한 질문이군요2015. 5. 13. 상가임대차법 개정으로 권리금 회수기회를 보호하도록 입법화 되었습니다.상가임차인은 임대차기간이 끝나기 6개월 전부터 임대차 종료 시까지  신규임차인을 구하여 임대인에게 소개함으로써 신규 임차인으로부터 권리금을 회수 할 수 있습니다. 이는 강행규정으로 상가임대차계약서상 권리금 포기 약정이 있더라도 신규임차인을 구하여 권리금을 회수할 수 있습니다.권리금 약정 포기를 하는 대신 임료 약정을 저렴하게 하였음에도 불구하고 권리금을 주장할 경우 도의적으로 문제가 될 수는 있으나 법적으로 권리금 포기약정은 효력이 없습니다. ​다만 1. 임차인이 주선한 신규임차인이 되려는 자가 보증금 또는 차임을 지급할 자력이 없는 경우2. 임차인이 주선한 신규임차인이 되려는 자가 임차인으로서의 의무를 위반할 우려가 있거나 그 밖에 임대차를 유지하기 어려운 상당한 사유가 있는 경우3. 임대차 목적물인 상가건물을 1년 6개월 이상 영리목적으로 사용하지 아니한 경우4. 임대인이 선택한 신규임차인이 임차인과 권리금 계약을 체결하고 그 권리금을 지급한 경우 ​5. 임차인이 3기의 차임액에 해당하는 금액에 이르도록 차임을 연체한 사실이 있는 경우6. 임차인이 거짓이나 그 밖의 부정한 방법으로 임차한 경우7. 서로 합의하여 임대인이 임차인에게 상당한 보상을 제공한 경우8. 임차인이 임대인의 동의 없이 목적 건물의 전부 또는 일부를 전대(轉貸)한 경우9. 임차인이 임차한 건물의 전부 또는 일부를 고의나 중대한 과실로 파손한 경우10. 임차한 건물의 전부 또는 일부가 멸실되어 임대차의 목적을 달성하지 못할 경우11. 임대인이 다음 각 목의 어느 하나에 해당하는 사유로 목적 건물의 전부 또는 대부분을 철거하거나 재건축하기 위하여 목적 건물의 점유를 회복할 필요가 있는 경우가. 임대차계약 체결 당시 공사시기 및 소요기간 등을 포함한 철

## 답변 카테고리

In [7]:
for i in range(3, 10, 5):
    print(qa_lines[i])

부동산, 건축

부동산, 건축



# 각 데이터를 리스트로 불러오기

In [8]:
qas = []     #질문, 답변 정보가 dict 형태로 들은 데이터
que_titles = []
que_texts = []
ans_texts = []
ans_categories = []
for i in range(len(qa_lines)):
    line = qa_lines[i]
    if i % 5 == 0:
        que_titles.append(line)
        qas.append({'index': int(i/5) + 1, 'que_title':line})
    if i % 5 == 1:
        que_texts.append(line)
    if i % 5 == 2:
        ans_texts.append(line)
    if i % 5 == 3:
        ans_categories.append(line)

In [9]:
qas[:2]

[{'index': 1, 'que_title': '권리금에 대한 질문요\n'},
 {'index': 2, 'que_title': '친구 전기세 가스비 청구\n'}]

In [10]:
print(len(que_texts), len(qas))

31605 31605


# 데이터를 dict의 list 형태로 저장 (que_text 제외)

In [11]:
for i in range(len(que_titles)):
    qas[i]['ans_text'] = ans_texts[i]
    qas[i]['ans_category'] = ans_categories[i]

In [12]:
print(type(qas), type(qas[0]))

<class 'list'> <class 'dict'>


In [13]:
qas[1]  # index, que_title, ans_text, ans_category로 구성되어 있는 dict
        # 질문 텍스트를 문장 토큰화 처리하여 질문만 추린 후 qas에 key-value로 추가한다

{'index': 2,
 'que_title': '친구 전기세 가스비 청구\n',
 'ans_text': '안녕하세요. 서울지방변호사회-네이버 지식iN 상담 변호사 최홍준 입니다.\u200b소송을 하셔야하는데....\u200b그 금액이 얼마인지에 따라서. 소송을 할지말지를 고민하셔야 할것같아요\u200b친구의 부주의 이기 때문에 받을수는 있습니다.\n',
 'ans_category': '부동산, 건축\n'}

# que_texts를 인덱스 부여 후 dict 형태로 저장

In [14]:
que_con_texts = []   # que_texts에 인덱스가 부여된 dict

for idx, text in enumerate(que_texts):
    que_con_texts.append({'index':idx + 1, 'que_text':text})

In [15]:
print(len(que_con_texts), '\n')

31605 



In [16]:
print(que_con_texts[0])

{'index': 1, 'que_text': '안녕하세요 권리금에 대해 질문이 있습니다.울산에서 조그만한 맥주집을 운영합니다.계약당시 계약서에 "단 권리금은 없다"라는 문구를 넣어서 계약을 했습니다.그러면 저는 계약 종료후 나갈때 새로운 임대인에게 권리금을 청구하지 못하는건가요?답변 부탁드립니다.꾸벅~\n'}


# 관련 카테고리 데이터만 필터링

## pandas로 빈출 키워드 분석

In [17]:
df = pd.DataFrame(qas)

In [18]:
df.columns

Index(['index', 'que_title', 'ans_text', 'ans_category'], dtype='object')

In [19]:
print(df['ans_category'].value_counts().head(50))

부동산, 건축\n        6635
전세\n             5259
월세\n             3472
임대차\n            1505
부동산\n            1346
부동산, 건축 민원\n     1226
경매\n             1220
매매\n             1132
계약\n             1118
건축학\n            1041
민법\n              752
법, 법률\n           600
민사소송\n            414
재판, 소송 절차\n       401
손해배상\n            295
법학, 법이론\n         287
형벌, 형집행\n         285
분양, 청약\n          263
형사사건\n            228
사기\n              222
신용, 파산\n          207
강력범죄\n            206
지식재산권\n           206
소비자관련법, 상법\n      196
제신고 민원\n          192
언론, 미디어 관련법\n     180
민사집행\n            176
교통 사고, 위반\n       165
등기\n              145
경범죄\n             139
노동법\n             137
폭행\n              136
절도\n              110
공갈, 협박\n           96
재산범죄\n             92
행정법\n              84
가족, 이혼\n           82
청소년관련법\n           77
의료, 법률, 금융\n       65
시설보수\n             50
부동산담보대출\n          49
금융상품담보대출\n         47
지역&플레이스\n          46
민원, 행정\n           45
헌법\n               37
교통민원\n    

## 빈출 키워드가 카테고리에 포함된 qa만 필터링

In [20]:
related_keywords = ['부동산', '건축', '전세', '월세', '임대차', '경매', '매매', '분양', '청약', '등기']

In [21]:
def keywords_match(qa):
    arr = []
    for kw in related_keywords:
        arr.append(int(kw in qa['ans_category']))
    return(arr)

In [22]:
keywords_match(qas[1000])

[1, 1, 0, 0, 0, 0, 0, 0, 0, 0]

In [23]:
filtered_qas = list(filter(lambda qa : sum(keywords_match(qa)), qas))

In [24]:
len(filtered_qas)

23308

## 필터링된 qa와 같은 index의 que_contexts를 필터링

In [25]:
# filtered_qas : 카테고리로 필터링된 qas,    23308
# que_con_texts : que_texts에 인덱스가 부여된 dict,    31605

In [25]:
filtered_ques = []
count = 0
for i in range(len(que_con_texts)):
    index = que_con_texts[i]['index']
    textt = que_con_texts[i]['que_text']
    if index == filtered_qas[count]['index']:
        filtered_ques.append({'index':index, 'que_text':textt})
        count += 1

In [26]:
print(filtered_ques[100]['index'])
print(filtered_ques[100]['que_text'])

101
우선 1남 1녀인데 1남인 저희 아버지께서 항상 제사랑 가족행사를 주관해왔습니다 근데 할머니가 돌아가신 후에 재산을 1녀인 고모한테 증여하시겠다고 하면 아버지가 받을 수 있는 재산은 없게 되는건가요?



In [27]:
print(filtered_qas[100]['index'])
print(filtered_qas[100]['que_title'])
print(filtered_qas[100]['ans_text'])
print(filtered_qas[100]['ans_category'])

101
자녀상속문제 궁금합니다

안녕하세요. 서울지방변호사회-네이버 지식iN 상담 변호사 김혜겸 입니다.​​​​할머님이 돌아가시기 전 유증(유언을 통한 증여)을 통해 만약 고모분께 재산을 증여하시는 의사를 밝히셨다면,아버님은 본래 상속분인 1/2 을 받을 수는 없고,다만 유류분반환청구를 통해  상속분의 절반을 유류분으로 청구할 수 있습니다. ​만약 할머님 생전에 아버님 혹은 고모분께 추가로 증여했던 재산(특별수익)이 있다면유류분산정과정에서 금액이 달라질수는 있습니다. ​​​​* 추가 댓글은 확인이 어렵습니다. 추가 문의사항 있으시면 네이버 expert 서비스를 이용하시면 추가상담이 가능하십니다. 감사합니다. ​​​

부동산, 건축



In [28]:
print(len(filtered_qas), len(filtered_ques))

23308 23308


# 질문 텍스트(que_texts) 전처리

In [29]:
for idx, text in enumerate(filtered_ques[::100]):
    print(idx*100, text, '\n')

0 {'index': 1, 'que_text': '안녕하세요 권리금에 대해 질문이 있습니다.울산에서 조그만한 맥주집을 운영합니다.계약당시 계약서에 "단 권리금은 없다"라는 문구를 넣어서 계약을 했습니다.그러면 저는 계약 종료후 나갈때 새로운 임대인에게 권리금을 청구하지 못하는건가요?답변 부탁드립니다.꾸벅~\n'} 

100 {'index': 101, 'que_text': '우선 1남 1녀인데 1남인 저희 아버지께서 항상 제사랑 가족행사를 주관해왔습니다 근데 할머니가 돌아가신 후에 재산을 1녀인 고모한테 증여하시겠다고 하면 아버지가 받을 수 있는 재산은 없게 되는건가요?\n'} 

200 {'index': 201, 'que_text': '부동산처분금지가처분 하는 방법을 알려주세요 법적으로 부부관계/ 실소유자 남자/\xa0부동산처분금지가처분 신청 원하는 사람 여자인터넷으로 하는 방법을 찾아 봤는데, 직접 관할법원가 가서 신청하는 것만 나와서 질문 드립니다.혹시 인터넷으로 신청하는 방법이 있다면 필요한 서류와 함께 안내 부탁드립니다.직접 관할법원가서 해야한다면 필요한 서류와 진행방법을 상세히 알려주시면 감사하겠습니다.만약 변호사와 함께 한다면 비용은 어느정도 들까요?내공냠냠, 정확하지 않은 정보, 홍보는 신고하겠습니다.\n'} 

300 {'index': 301, 'que_text': '갑자기 전등이 꺼졌어요\xa0 전자제품은 않 꺼진걸론 봐선 전등 문재인것 같은데1방은 전기가 켜져요\n'} 

400 {'index': 401, 'que_text': '올해 어머니가 돌아가셨고 상속인은 누나와 저 둘 입니다5년전 아버지가 돌아가실때 어머니랑 저 앞으로 원룸건물5대5 상속 받았고 누나는 시골 빌라를 상속 받았습니다 지금 원룸은 명의는 어머니랑 저 둘로 되어 있지만 실질적으로 어머니가 월세를 다 관리하시고 계약서에 통장도 어머니 통장으로 되어있습니다 이상태에서 상속시 아버지 돌아가시고 5년 동안의 원룸 수익의 반을 기여도에 포함 할 수 있나요?\n'} 

500 {'in

## 점 여러개 => 1개

In [30]:
text_1 = filtered_ques[13700]['que_text']
text_1

'지금 살고 있는 집 2년 계약 후 묵시적으로 2년째 살아오는데 집주인이 계약서를 작성하자고 합니다. 전세금은 변동 없습니다... 이런경우 연장계약으로 가야하나요? 아니면 일정을 새로 다시 잡아야하나요? ㅜㅜ 확정일자 다시 잡아야하나여??\n'

In [31]:
text_re_1 = re.sub('\.{2,20}', '.', text_1)
text_re_1

'지금 살고 있는 집 2년 계약 후 묵시적으로 2년째 살아오는데 집주인이 계약서를 작성하자고 합니다. 전세금은 변동 없습니다. 이런경우 연장계약으로 가야하나요? 아니면 일정을 새로 다시 잡아야하나요? ㅜㅜ 확정일자 다시 잡아야하나여??\n'

## 빈 공간 여러 개 => 1개

In [32]:
text_2 = filtered_ques[23100]['que_text']
text_2

'아파트를 매도후 양도소득세를 직접 신고하려 하는데..간단치가 안네요.. 강남의 아파트구요...\xa0\xa0 31평형(84.97m) 입니다.2년이상 거주했구요... 4년 보유하여 비과세 요건은 갖추었습니다.\xa0매수: 2003년 10월 \xa0548,500,000매도: 2007년 11월\xa0 737,000,000\xa0기타경비는 등록세 취득세 법무사 수수료 : 10,247,700중개수수료: 매수시 2,100,000\xa0 매도시 3,680,000그리고 매수시 대출을 받으며 들어간 근저당설정비와 법무사 수수료는 인정이 안되나요?인정이 된다면 그 비용은 813,300 입니다.양도소득세 계산 부탁드립니다.\n'

In [33]:
text_repl_2 = text_2.replace('\xa0', ' ')
text_re_2 = re.sub('\s{2,}', '', text_repl_2)
text_re_2

'아파트를 매도후 양도소득세를 직접 신고하려 하는데..간단치가 안네요.. 강남의 아파트구요...31평형(84.97m) 입니다.2년이상 거주했구요... 4년 보유하여 비과세 요건은 갖추었습니다. 매수: 2003년 10월548,500,000매도: 2007년 11월737,000,000 기타경비는 등록세 취득세 법무사 수수료 : 10,247,700중개수수료: 매수시 2,100,000매도시 3,680,000그리고 매수시 대출을 받으며 들어간 근저당설정비와 법무사 수수료는 인정이 안되나요?인정이 된다면 그 비용은 813,300 입니다.양도소득세 계산 부탁드립니다.\n'

## 띄어쓰기

In [34]:
text_3 = filtered_ques[11100]['que_text']
text_3

'한달에15만원씩내고 상가를쓰고있습니다근데 갑자기 주인이 일주일에 하루쓰는거아니었냐 개소리를합니다..계약서에 명시하지도않았구요이거 어떻게해야하나요계약위반으로 고소할수있나요계약서에 매일쓴다고 적진않았는데 보통 안써도 암묵적으로 월세내면 제가 언제든 사용하던말던 상관없지않나요?자긴 저번 계약자라 그랬다면서 우기네요\n'

### 본문의 띄어쓰기, 빈 공간 제거

In [35]:
text_no_space_3 = re.sub('\s', '', text_3)
text_no_space_3

'한달에15만원씩내고상가를쓰고있습니다근데갑자기주인이일주일에하루쓰는거아니었냐개소리를합니다..계약서에명시하지도않았구요이거어떻게해야하나요계약위반으로고소할수있나요계약서에매일쓴다고적진않았는데보통안써도암묵적으로월세내면제가언제든사용하던말던상관없지않나요?자긴저번계약자라그랬다면서우기네요'

### 빈 공간 없는 텍스트를 다시 띄어쓰기

In [36]:
text_re_3 = spacing(text_no_space_3)
text_re_3

'한 달에 15만원씩 내고 상가를 쓰고 있습니다 근데 갑자기 주인이 일주일에 하루 쓰는 거 아니었냐 개소리를 합니다.. 계약서에 명시하지도 않았구요 이거 어떻게 해야 하나요 계약 위반으로 고소할 수 있나요 계약서에 매일 쓴다고 적진 않았는데 보통 안 써도 암묵적으로 월세 내면제가 언제든 사용하던 말던 상관없지 않나요?자 긴 저번 계약자라 그랬다면서 우기네요'

## 이모티콘 제거

In [37]:
text_5 = filtered_ques[3800]['que_text']
text_5

'안녕하세요제가 독서실을 하는데요 바로 옆에 1층건물을 허물고 4층짜리 건물을 짓는다고 하는데아직 공사 시작은 안했구요 5월 1일 시작으로 되어있더라구요ㅜ바로 옆쪽에서 독서실을 하는 저희로선 소음으로 인해 엄청난 타격을 입을 거 같은데어떻게 해야 되나 막막합니다ㅜㅠㅠ 하루종일 소리가 장난 아닐텐데 회원한테도 피해가 갈거구요...해결 할 수 있는 방법이 있나해서 질문합니다! 저희가 할 수 있는 조치가 있을까요??\n'

In [38]:
text_re_5 = re.sub('[ㅠㅜㅡ^]{2,}|[~]{2,}|[ㅎㅋ]{2,}', '', text_5)
text_re_5

'안녕하세요제가 독서실을 하는데요 바로 옆에 1층건물을 허물고 4층짜리 건물을 짓는다고 하는데아직 공사 시작은 안했구요 5월 1일 시작으로 되어있더라구요ㅜ바로 옆쪽에서 독서실을 하는 저희로선 소음으로 인해 엄청난 타격을 입을 거 같은데어떻게 해야 되나 막막합니다 하루종일 소리가 장난 아닐텐데 회원한테도 피해가 갈거구요...해결 할 수 있는 방법이 있나해서 질문합니다! 저희가 할 수 있는 조치가 있을까요??\n'

## 문장 토큰화

In [39]:
text4 = filtered_ques[4200]['que_text']
text4

"재개발 구역 세입자입니다. 세입자 보상비 관련해 여쭤보려합니다. 2017년 사업시행 허가가 떨어졌구요..아직 이주.철거명령은 내리지 않은 상태입니다.이주비 보상은 2007년2월말 이전 전입신고된 가구만 나온다고 합니다.재개발 조합에서는 2016년에 '보상이 해당되는 세입자'는 접수신청을 하라고 공고했습니다.(저는 해당 사항이 없다 생각해 그당시 접수를 못했습니다)저희는 A재개발구역에서 2001년부터 2007년3월18일까지 거주했고(2001년 전입신고 했구요)동일한 A재개발지구로 이사해 2007년3월 19일에 전입신고하구..2017년6월 현재 까지 거주하고 있습니다.1.세입자 보상비와..임대주택 신청이 가능한지요?2.만약 보상이된다면 2016년 당시 신청대상이 아닐꺼라 생각해 신청을 못했는데...지금도 신청가능한건지요\n"

In [40]:
text_re_4 = kss.split_sentences(text4)
for idx, text in enumerate(text_re_4):
    print(idx+1, '-', text)

1 - 재개발 구역 세입자입니다.
2 - 세입자 보상비 관련해 여쭤보려합니다.
3 - 2017년 사업시행 허가가 떨어졌구요..
4 - 아직 이주.철거명령은 내리지 않은 상태입니다.
5 - 이주비 보상은 2007년2월말 이전 전입신고된 가구만 나온다고 합니다.
6 - 재개발 조합에서는 2016년에 '보상이 해당되는 세입자'는 접수신청을 하라고 공고했습니다.
7 - (저는 해당 사항이 없다 생각해 그당시 접수를 못했습니다)저희는 A재개발구역에서 2001년부터 2007년3월18일까지 거주했고(2001년 전입신고 했구요)동일한 A재개발지구로 이사해 2007년3월 19일에 전입신고하구..2017년6월 현재 까지 거주하고 있습니다.
8 - 1.세입자 보상비와..임대주택 신청이 가능한지요?
9 - 2.만약 보상이된다면 2016년 당시 신청대상이 아닐꺼라 생각해 신청을 못했는데...지금도 신청가능한건지요


## 1~4 일괄 적용

In [41]:
def clean_text(text):
    
    text1 = re.sub('\.{2,20}', '.', text)   # 점 여러개 제거
    text2 = text1.replace('\xa0', ' ')   # 빈 공간 여러개 제거
    text3 = text2.strip()
    text4 = re.sub('[ㅠㅜㅡ^]{2,}|[~]{2,}|[ㅎㅋ]{2,}', '', text3)  # 이모티콘 제거
    text5 = re.sub('\s', '', text4)  
    text6 = spacing(text5)   # 다시 띄어쓰기
    text7 = kss.split_sentences(text6)   # 문장 토큰화
    
    return text7

In [42]:
print(filtered_ques[100], '\n')

{'index': 101, 'que_text': '우선 1남 1녀인데 1남인 저희 아버지께서 항상 제사랑 가족행사를 주관해왔습니다 근데 할머니가 돌아가신 후에 재산을 1녀인 고모한테 증여하시겠다고 하면 아버지가 받을 수 있는 재산은 없게 되는건가요?\n'} 



In [43]:
print(len(filtered_ques))

23308


In [44]:
cleaned_que_texts = []
for que in filtered_ques:
    cleaned_que = clean_text(que['que_text'])
    cleaned_que_texts.append(cleaned_que)

In [45]:
len(cleaned_que_texts)

23308

In [46]:
for q in cleaned_que_texts[::1000]:
    print(q, '\n')

['안녕하세요', '권리금에 대해 질문이 있습니다.', '울산에서 조그만한 맥주집을 운영합니다.', '계약 당시 계약서에 "단 권리금은 없다"라는 문구를 넣어서 계약을 했습니다.', '그러면 저는 계약 종료 후 나갈 때 새로운 임대인에게 권리금을 청구하지 못하는 건가요?', '답변 부탁드립니다.', '꾸벅~'] 

['수익형 호텔 질문 드립니다', '분양대행사로부터 계약 당시 추후에 분양사로부터 바이백된다고 하여 바이백 안심증서를 받고 계약하였는데 분양사에서는 제가 가지고 있는 바이백증서는 효력이 없다고 합니다', '바이백 증서에 분양사직인도 찍여 있고 하는데 말입니다', '분양사기 아닌가요', '계약 해지 사유가 되는 지 문의 드립니다'] 

['최근 6년 동안 수시 결과를 보았을 때 제일 적게 추가 합격이 된 인원은 4명이 고 작년에는 6명 충원 합격했는데 저는 가능성이 얼마나 될까요?'] 

['서울 강서구 화곡4동811-34번에 위치한 12세대 연립입니다.', '대지는 153-6평 정도?', '기존 세대 중 6세대 정도가 재입주한다고 하였을 때 (지하층 4세대 제외) 사업성이 있은까요?', '단독으로 아파트 형태로 건축은 가능한지요?', '문의 드립니다.', '더운 날씨에 고생하세요'] 

['저희 직원이 현재, 경력과 사이버대학 관련 학과 이수부분으로 자격증을 취득 시에 초급기사 취득이 가능합니다.', '그런데, 기술인협회에 문의했더니 초급을 취득한 그날로부터 무조건 10년이 지나야 중급 신청이 가능하다고 안내를 받았다고 하네요.', '그 말이 맞다면 초급기사를 아예 따놓지 않고, 더 경력을 쌓아서 중급을 취득하는 게 중급 자 격 획득에 시간 절약이 아닌가요? ?', '직원이 알고 있는 부분이 맞는 건지 확인하고 싶어서 질문합니다.', '건축기사 수첩 취득에 관련해서 좀자 세하게 급수별로 안 내 부탁드립니다.', '그리고 한 단계씩 높은 등급으로 변경 신청할 때 처음 말한 대로년 수 제한이 있는지도 알려주세요.', '감사합니다.'] 

['지금 성남에

# 질문 텍스트 문장들에서 의미있는 질문 문장 추출

## ?가 질문 문장에 포함되었는지 여부로 qas를 filtering

In [47]:
filtered_qas[0]

{'index': 1,
 'que_title': '권리금에 대한 질문요\n',
 'ans_text': '안녕하세요. 계약, 부동산, 임대차 등 재산분쟁을 주요 업무로 하는서울지방변호사회-네이버 지식iN 상담 변호사 이동영 입니다.\u200b상가임대차에 관한 질문이군요2015. 5. 13. 상가임대차법 개정으로 권리금 회수기회를 보호하도록 입법화 되었습니다.상가임차인은 임대차기간이 끝나기 6개월 전부터 임대차 종료 시까지  신규임차인을 구하여 임대인에게 소개함으로써 신규 임차인으로부터 권리금을 회수 할 수 있습니다. 이는 강행규정으로 상가임대차계약서상 권리금 포기 약정이 있더라도 신규임차인을 구하여 권리금을 회수할 수 있습니다.권리금 약정 포기를 하는 대신 임료 약정을 저렴하게 하였음에도 불구하고 권리금을 주장할 경우 도의적으로 문제가 될 수는 있으나 법적으로 권리금 포기약정은 효력이 없습니다. \u200b다만 1. 임차인이 주선한 신규임차인이 되려는 자가 보증금 또는 차임을 지급할 자력이 없는 경우2. 임차인이 주선한 신규임차인이 되려는 자가 임차인으로서의 의무를 위반할 우려가 있거나 그 밖에 임대차를 유지하기 어려운 상당한 사유가 있는 경우3. 임대차 목적물인 상가건물을 1년 6개월 이상 영리목적으로 사용하지 아니한 경우4. 임대인이 선택한 신규임차인이 임차인과 권리금 계약을 체결하고 그 권리금을 지급한 경우 \u200b5. 임차인이 3기의 차임액에 해당하는 금액에 이르도록 차임을 연체한 사실이 있는 경우6. 임차인이 거짓이나 그 밖의 부정한 방법으로 임차한 경우7. 서로 합의하여 임대인이 임차인에게 상당한 보상을 제공한 경우8. 임차인이 임대인의 동의 없이 목적 건물의 전부 또는 일부를 전대(轉貸)한 경우9. 임차인이 임차한 건물의 전부 또는 일부를 고의나 중대한 과실로 파손한 경우10. 임차한 건물의 전부 또는 일부가 멸실되어 임대차의 목적을 달성하지 못할 경우11. 임대인이 다음 각 목의 어느 하나에 해당하는 사유로 목적 건물의 전부 또는 대부분을 철거

In [48]:
cleaned_que_texts[0]

['안녕하세요',
 '권리금에 대해 질문이 있습니다.',
 '울산에서 조그만한 맥주집을 운영합니다.',
 '계약 당시 계약서에 "단 권리금은 없다"라는 문구를 넣어서 계약을 했습니다.',
 '그러면 저는 계약 종료 후 나갈 때 새로운 임대인에게 권리금을 청구하지 못하는 건가요?',
 '답변 부탁드립니다.',
 '꾸벅~']

In [49]:
cleaned_que_texts_with_i = []

for idx, qa in enumerate(filtered_qas):
    index = qa['index']
    arrr = cleaned_que_texts[idx]
    cleaned_que_texts_with_i.append({'index':index, 'que_arr':arrr})

In [50]:
len(cleaned_que_texts_with_i)

23308

## 물음표가 포함된 질문 문장만 저장

In [51]:
sent_with_que_marks = []
for idx, arr in enumerate(cleaned_que_texts_with_i):
    index = cleaned_que_texts_with_i[idx]['index']
    arr = cleaned_que_texts_with_i[idx]['que_arr']
    que_mark = []
    for i in range(len(arr)):
        if '?' in arr[i]:
            que_mark.append(arr[i])
    if que_mark == []:
        pass
    sent_with_que_marks.append({'index':index, 'ques':que_mark})

In [52]:
ques_filtered = list(filter(lambda x: len(x['ques']) != 0 ,sent_with_que_marks))

### 질문 문장들 중 가장 긴 문장을 best_question으로 저장

In [53]:
for row in ques_filtered:
    ques = row['ques']
    ques.sort(key=len, reverse=True)
    longest_que = ques[0]
    row['best_question'] = longest_que

In [54]:
ques_filtered[9]['ques']

['구역 지정? 혹은 조합설립? 질문4: 마지막으로 조정 대상지역이긴 한데 다른 지역에 미니 재건축 대상 아파트를 하나 소유하고 있습니다.',
 '질문3:보유하게 되는 아파트 2채 중에 한 채를 언제까지 매도해야지 매수인한테도 조합원 자격이 생기게 된 느지요?',
 '질문1: 조정 대상지역이 아닐 경우에는 같은 재건축 단지 내에 2채 보유 시에는 입주권도 2개인 지요?',
 '혹시 그렇다면 재건축의 구역지정 혹은 조합설립 중에어는 단계가 기준이 되는지요?',
 '질문2: 조정 대상지역인 경우에는 입주권이 1개로 제한되는지요?',
 '위에서 문의한 재건축하고 상관없이 조합원 대상이 되는 건 지요?']

### best_question의 길이를 que_len으로 저장

In [55]:
for row in ques_filtered:
    que_len = len(row['best_question'])
    row['que_len'] = que_len

In [56]:
len(ques_filtered)

17817

In [57]:
ques_filtered[0] # index, ques, best_question, que_len

{'index': 1,
 'ques': ['그러면 저는 계약 종료 후 나갈 때 새로운 임대인에게 권리금을 청구하지 못하는 건가요?'],
 'best_question': '그러면 저는 계약 종료 후 나갈 때 새로운 임대인에게 권리금을 청구하지 못하는 건가요?',
 'que_len': 48}

## best_question의 길이 분포

In [58]:
df = pd.DataFrame(ques_filtered)
df.shape

(17817, 4)

In [59]:
df.columns

Index(['index', 'ques', 'best_question', 'que_len'], dtype='object')

In [60]:
df['que_len'].value_counts()

40     261
44     245
37     245
33     245
27     244
      ... 
309      1
293      1
261      1
245      1
431      1
Name: que_len, Length: 373, dtype: int64

## ques_filtered => df => excel 포맷 변환

In [68]:
df.to_excel('../Data/kin/que_sentences.xlsx')

## 엑셀 파일 => 답변 길이 15~160으로 필터링 => 엑셀 불러오기

In [61]:
filtered_excel_file = pd.read_excel('../../Data/Naver_kin_QA(estate)/que_sentences_v1.2.xlsx', engine='openpyxl')

In [62]:
filtered_excel_file.shape

(16465, 5)

In [63]:
del filtered_excel_file[0]

In [64]:
filtered_excel_file.columns

Index(['index', 'ques', 'best_question', 'que_len'], dtype='object')

In [65]:
que_filter = filtered_excel_file.values.tolist()

In [66]:
len(que_filter)

16465

## 불러온 엑셀 데이터 => dict 형태로 바꾸기

In [67]:
que_filter[0]

[1,
 "['그러면 저는 계약 종료 후 나갈 때 새로운 임대인에게 권리금을 청구하지 못하는 건가요?']",
 '그러면 저는 계약 종료 후 나갈 때 새로운 임대인에게 권리금을 청구하지 못하는 건가요?',
 48]

In [68]:
print(que_filter[0][0], que_filter[0][2], que_filter[0][3])

1 그러면 저는 계약 종료 후 나갈 때 새로운 임대인에게 권리금을 청구하지 못하는 건가요? 48


In [69]:
ques_filter = []
for que in que_filter:
    index = que[0]
    question = que[2]
    que_len = que[3]
    ques_filter.append({'index':index, 'que':question, 'que_len':que_len})

In [70]:
ques_filter[0]

{'index': 1,
 'que': '그러면 저는 계약 종료 후 나갈 때 새로운 임대인에게 권리금을 청구하지 못하는 건가요?',
 'que_len': 48}

# best_question & 법령 문서 => TF-IDF

## 샘플 질문

In [73]:
sample_que = ques_filter[2214]['que']
sample_que

'저희가 농지 400평을 구매했는데 그 농지 위에 9평짜리 컨테이너와 창고식의 가 건물을 건축했는데 농지 안에 있으니까 문제가 되서 50평 정도 대지로 변경을 하려고 하는데 창고하고 컨테이너하고 다 없애야 대지로 변경이 가능한가요?'

## 토크나이저

In [74]:
from konlpy.tag import Mecab
m = Mecab()
tokenized = m.morphs(sample_que)

In [75]:
print(tokenized)

['저희', '가', '농지', '400', '평', '을', '구매', '했', '는데', '그', '농지', '위', '에', '9', '평', '짜리', '컨테이너', '와', '창', '고식', '의', '가', '건물', '을', '건축', '했', '는데', '농지', '안', '에', '있', '으니까', '문제', '가', '되', '서', '50', '평', '정도', '대지', '로', '변경', '을', '하', '려고', '하', '는데', '창고', '하고', '컨테이너', '하', '고', '다', '없애', '야', '대지', '로', '변경', '이', '가능', '한가요', '?']


## context 데이터 구성 (모든 법령)

In [76]:
with open('../../Data/부동산법률_전처리_json/preprocess_statue.json', 'r', encoding='utf-8') as f:
    json_data = json.load(f)

In [77]:
json_data[0] # 각 법령 문서

{'index': 1,
 'title': '상가건물 임대차보호법',
 'enforcement_date': ' 2020. 11. 1.',
 'statue_class': '법률',
 'statue_number': '제17471호',
 'provisions': [{'info': '제1조(목적)',
   'text_show': '제1조(목적)\n이 법은 상가건물 임대차에 관하여 「민법」에 대한 특례를 규정하여 국민 경제생활의 안정을 보장함을 목적으로 한다.\n[전문개정 2009. 1. 30.]\n',
   'text_model': '이 법은 상가건물 임대차에 관하여 「민법」에 대한 특례를 규정하여 국민 경제생활의 안정을 보장함을 목적으로 한다.\n'},
  {'info': '제2조(적용범위)',
   'text_show': '제2조(적용범위)\n① 이 법은 상가건물(제3조제1항에 따른 사업자등록의 대상이 되는 건물을 말한다)의 임대차(임대차 목적물의\n주된 부분을 영업용으로 사용하는 경우를 포함한다)에 대하여 적용한다. 다만, 제14조의2에 따른 상가건물임대차위원회의 심\n의를 거쳐 대통령령으로 정하는 보증금액을 초과하는 임대차에 대하여는 그러하지 아니하다. <개정 2020. 7. 31.>\n② 제1항 단서에 따른 보증금액을 정할 때에는 해당 지역의 경제 여건 및 임대차 목적물의 규모 등을 고려하여 지역별로 구분하\n여 규정하되, 보증금 외에 차임이 있는 경우에는 그 차임액에 「은행법」에 따른 은행의 대출금리 등을 고려하여 대통령령으로 정\n하는 비율을 곱하여 환산한 금액을 포함하여야 한다.<개정 2010. 5. 17.>\n③ 제1항 단서에도 불구하고 제3조, 제10조제1항, 제2항, 제3항 본문, 제10조의2부터 제10조의9까지의 규정 및 제19조는 제\n1항 단서에 따른 보증금액을 초과하는 임대차에 대하여도 적용한다.<신설 2013. 8. 13., 2015. 5. 13., 2020. 9. 29.>\n[전문개정 2009. 1. 30.]\n',
   'text_mod

In [78]:
data = []  # 모든 법령 문서(138개)를 문장 토큰화하여 모든 문장을 저장

for j in json_data:
    doc = j['provisions']
    for d in doc:
        para = d['text_model']
        sentence = para     # 각 조문을 data로 저장
                            # naver_kin_preprocess_v1.5_201229 와 다른 방법
        data.append(sentence)

In [79]:
len(data)

9580

## TF-IDF 설정

In [80]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [81]:
tfidf = TfidfVectorizer(analyzer=str.split
                        , encoding="utf-8"
                        , stop_words="korean"
                        , ngram_range=(1, 3))

In [82]:
tfidf_context = tfidf.fit_transform(data)

In [83]:
tfidf_question = tfidf.transform([sample_que])

## consine_similarity vector 구하기

In [84]:
sim = cosine_similarity(tfidf_question, tfidf_context)
print(sim)

[[0.         0.00122807 0.00330482 ... 0.         0.         0.        ]]


## pytorch topk

In [85]:
import torch

In [86]:
sim_vec = torch.tensor(sim)

In [87]:
top_ans_list = torch.topk(sim_vec, 3)[1].tolist()[0]
top_ans_list

[1172, 1209, 1219]

In [88]:
sample_que

'저희가 농지 400평을 구매했는데 그 농지 위에 9평짜리 컨테이너와 창고식의 가 건물을 건축했는데 농지 안에 있으니까 문제가 되서 50평 정도 대지로 변경을 하려고 하는데 창고하고 컨테이너하고 다 없애야 대지로 변경이 가능한가요?'

In [89]:
for idx in top_ans_list:
    print(data[idx], ' / ',sim[0][idx], '\n')

시장ㆍ군수ㆍ자치구구청장, 한국농어촌공사, 그 밖에 대통령령으로 정하는 자는 농지이용계획에 따라 농지 이용을 증진하기 위하여 다음 각 호의 어느 하나에 해당하는 사업을 시행할 수 있다. <개정 2008,12.2.농지의 매매ㆍ교환ㆍ분합 등에 의한 농지 소유권 이전을 촉진하는 사업. 농지의 장기 임대차, 장기 사용대차에 따른 농지 임차권 설정을 촉진하는 사업. 위탁경영을 촉진하는 사업. 농업인이나 농업법인이 농지를 공동으로 이용하거나 집단으로 이용하여 농업경영을 개선하는 농업 경영체 육성사업
  /  0.1742151232081719 

시ㆍ구ㆍ읍ㆍ면의 장은 농지 소유 실태와 농지 이용 실태를 파악하여 이를 효율적으로 이용하고 관리하기 위하여 대통령령으로 정하는 바에 따라 농지원부를 작성하여 갖추어 두어야 한다.  . 농지법. 시ㆍ구ㆍ읍ㆍ면의 장은 제1항에 따른 농지원부를 작성ㆍ정리하거나 농지 이용 실태를 파악하기 위하여 필요하면 해당 농지소유자에게 필요한 사항을 보고하게 하거나 관계 공무원에게 그 상황을 조사하게 할 수 있다.시ㆍ구ㆍ읍ㆍ면의 장은 농지원부의 내용에 변동사항이 생기면 그 변동사항을 지체 없이 정리하여야 한다.제1항의 농지원부에 적을 사항을 전산정보처리조직으로 처리하는 경우 그 농지원부 파일은 제1항에 따른 농지원부로 본다.농지원부의 서식ㆍ작성ㆍ관리와 전산정보처리조직 등에 필요한 사항은 농림축산식품부령으로 정한다.
  /  0.1690121642302464 

다음 각 호의 어느 하나에 해당하는 자는 . 이하의 징역 또는 5천만원 이하의 벌금에 처한다.제6조에 따른 농지 소유 제한이나 제7조에 따른 농지 소유 상한을 위반하여 농지를 소유할 목적으로 거짓이나 그 밖의 부정한 방법으로 제8조제1항에 따른 농지취득자격증명을 발급받은 자. 제32조제. 또는 제2항을 위반한 자. 제36조제1항에 따른 농지의 타용도 일시사용허가를 받지 아니하고 농지를 다른 용도로 사용한 자. 제40조제1항을 위반하여 전용된 토지를 승인 없이 다른 목적으로 사용한 자
  /  0

## 모든 질문에 1~6 과정 적용

In [91]:
print(len(ques_filter), '\n', ques_filter[0])

16465 
 {'index': 1, 'que': '그러면 저는 계약 종료 후 나갈 때 새로운 임대인에게 권리금을 청구하지 못하는 건가요?', 'que_len': 48}


### 질문만으로 구성된 리스트 생성

In [92]:
questions = [que['que'] for que in ques_filter]

In [93]:
print(len(questions), '\n', questions[0])

16465 
 그러면 저는 계약 종료 후 나갈 때 새로운 임대인에게 권리금을 청구하지 못하는 건가요?


### 각 질문들 토큰화하여 리스트의 리스트로 저장

In [39]:
from konlpy.tag import Mecab
m = Mecab()

In [40]:
q_tokens = []

for idx, q in enumerate(questions):
    tokens = []
    try:
        tokens = m.morphs(q)
        q_tokens.append(tokens)
    except:
        print(idx)
        pass

1142
3137
3482


In [41]:
len(q_tokens)

16462

### questions들 중에서 tokenizing이 안되는 데이터 제외

In [94]:
print(questions[1142])
print(questions[3137])
print(questions[3482])

nan
nan
nan


In [95]:
del questions[3482]
del questions[3137]
del questions[1142]

In [96]:
len(questions)

16462

### TF-IDF context, question

In [97]:
len(data)

9580

In [98]:
tfidf_context = tfidf.fit_transform(data)  # context

In [99]:
tfidf_que_list = []  # question list

for idx, q in enumerate(questions):
    tf_idf = tfidf.transform([q])
    tfidf_que_list.append(tf_idf)

In [100]:
tfidf_que_list[0]

<1x104217 sparse matrix of type '<class 'numpy.float64'>'
	with 9 stored elements in Compressed Sparse Row format>

In [101]:
len(tfidf_que_list)

16462

### cosine 유사도 구하기

In [102]:
sim_list = []    # cosine similarity list

for q in tfidf_que_list:
    sim = cosine_similarity(q, tfidf_context)
    sim_list.append(sim)

In [103]:
len(sim_list)

16462

### top5 답변 구하기

In [52]:
top5_idx = []

for sim in sim_list:
    sim_vec = torch.tensor(sim)
    top5_ans = torch.topk(sim_vec, 5)[1].tolist()[0]
    top5_idx.append(top5_ans)

In [53]:
len(top5_idx)

16462

In [90]:
questions[1114]

NameError: name 'questions' is not defined

In [55]:
top5_idx[4997]

[4999, 9261, 6967, 1990, 4262]

In [56]:
for idx in top5_idx[4997]:
    print(data[idx], ' / ', sim_list[4997][0][idx], '\n')

법 제22조제3항에 따른 특별시ㆍ광역시ㆍ특별자치시ㆍ특별자치도 도시ㆍ군기본계획의 공고는 해당 특별시ㆍ광역시ㆍ특별자치시ㆍ특별자치도의 공보와 인터넷 홈페이지에 게재하는 방법으로 하며, 관계 서류의 열람기간은 3. 이상으로 해야 한다. 
  /  0.09383871657797388 

법 제39조제. 각 호의 시설의 설치자 또는 관리자는 댐등의 상류에 있는 하천의 하상 매몰, 수위상승 또는 댐등으로의 유입량의 증가로 재해발생의 우려가 있는때에는 그 상황에 따라 필요한 조치를 해야 한다.댐등의 설치자 또는 관리자는 댐등의 하류에 유량이 현저히 증가하거나 물을 흘려보낼 곳의 수위 또는 해면이 상승하여 댐등에 가두어 둔 물을 방류하는 경우 재해가 발생할 우려가 있을 때에는 적절한 방식으로 그 증가유량을 조절하여 재해가 발생하지 않도록 해야 한다.댐등의 설치자 또는 관리자는 댐등으로 인한 재해발생을 예방하기 위하여 다음 각 호의 조치를 해야 한다.사이렌 및 스피커 방송 등을 위한 경보시설의 사전 설치 및 운용. 제2항에 따라 방류하려는 때의 경보. 제2항에 따라 방류하려는 때의 관계 기관에 대한 통지 및 그 밖에 필요한 조치. 제2항에 따른 방류시기 및 그 밖에 필요한 사항의 공고제4. 삭제 
  /  0.09178759168460036 

다음 각 호의 어느 하나에 해당하는 배당소득에 대해서는 다음 각 호에 따른 날에 그 소득을 지급한 것으로 보아 소득세를 원천징수한다.의제배당 제46조제. 또는 제5호에 규정된 날. 출자공동사업자의 배당소득으로서 과세기간 종료 후 3개월이 되는 날까지 지급하지 아니한 소득 과세기간 종료 후 3개월이 되는 날. 「조세특례제한법」 제100조의18제1항에 따라 배분받는 소득으로서 해당 동업기업의 과세기간 종료 후 3개월이 되는 날까지지급하지 아니한 소득 해당 동업기업의 과세기간 종료 후 3개월이 되는 날. 삭제. 그 밖의 배당소득 제4. 각 호에 규정된 날 
  /  0.08207693856028524 

지역ㆍ지구등은 다음 각 호의 어느 하나

### top5 답변 리스트로 저장

In [57]:
top5_idx[0]

[1712, 11, 6967, 1570, 3178]

In [58]:
top5_answers = []

for i, index in enumerate(top5_idx):
    candidates = []
    candidates.append(questions[i])
    for n in index:
        ans = data[n]
        score = sim_list[i][0][n]
        candidates.append(ans)
        candidates.append(score)
    top5_answers.append(candidates)

In [59]:
top5_answers[0]

['그러면 저는 계약 종료 후 나갈 때 새로운 임대인에게 권리금을 청구하지 못하는 건가요?',
 '법 제23조제2항에 따른 임대차 또는 사용대차의 종료 명령은 별지 제13호의2서식에 따른다.종료 명령을 받은 임차인 또는 사용대차인은 그 종료 명령을 받은 날부터 3개월 이내에 해당 계약을 종료하여야 한다.시장ㆍ군수ㆍ구청장은 종료 명령을 한 경우에는 임대인에게 그 사실을 즉시 알려야 한다.\n',
 0.2414240130464516,
 '임대인은 임대차기간이 끝나기 6개월 전부터 임대차 종료 시까지 다음 각 호의 어느 하나에 해당하는 행위를 함으로써 권리금 계약에 따라 임차인이 주선한 신규임차인이 되려는 자로부터 권리금을 지급받는 것을 방해하여서는 아니 된다. 다만, 제10조제. 각 호의 어느 하나에 해당하는 사유가 있는 경우에는 그러하지 아니하다.임차인이 주선한 신규임차인이 되려는 자에게 권리금을 요구하거나 임차인이 주선한 신규임차인이 되려는 자로부터 권리금을 수수하는 행위. 임차인이 주선한 신규임차인이 되려는 자로 하여금 임차인에게 권리금을 지급하지 못하게 하는 행위. 임차인이 주선한 신규임차인이 되려는 자에게 상가건물에 관한 조세, 공과금, 주변 상가건물의 차임 및 보증금, 그 밖의 부담에 따른 금액에 비추어 현저히 고액의 차임과 보증금을 요구하는 행위. 그 밖에 정당한 사유 없이 임대인이 임차인이 주선한 신규임차인이 되려는 자와 임대차계약의 체결을 거절하는 행위. 다음 각 호의 어느 하나에 해당하는 경우에는 제1항제4호의 정당한 사유가 있는 것으로 본다.임차인이 주선한 신규임차인이 되려는 자가 보증금 또는 차임을 지급할 자력이 없는 경우. 임차인이 주선한 신규임차인이 되려는 자가 임차인으로서의 의무를 위반할 우려가 있거나 그 밖에 임대차를 유지하기 어려운 상당한 사유가 있는 경우. 임대차 목적물인 상가건물을 . 6개월 이상 영리목적으로 사용하지 아니한 경우. 임대인이 선택한 신규임차인이 임차인과 권리금 계약을 체결하고 그 권리금을 지급한 경우. 임대인이 제

In [60]:
len(top5_answers)

16462

### top5 답변 리스트 excel로 저장

In [61]:
ddf = pd.DataFrame(top5_answers)

In [63]:
ddf.to_excel('../Data/kin/qna_top5(2).xlsx')

### excel => column 이름 변경 후 저장 => 다시 불러오기

In [64]:
fixed_qna = pd.read_excel('../Data/kin/qna_top5(2).xlsx')

In [65]:
fixed_qna.columns

Index(['index', 'question', 'ans1', 'score1', 'ans2', 'score2', 'ans3',
       'score3', 'ans4', 'score4', 'ans5', 'score5'],
      dtype='object')

# KoELECTRA pipeline

In [66]:
from transformers import ElectraTokenizer, ElectraForQuestionAnswering, pipeline
from pprint import pprint

In [67]:
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-small-v2-distilled-korquad-384")
model = ElectraForQuestionAnswering.from_pretrained("monologg/koelectra-small-v2-distilled-korquad-384")

In [68]:
qa = pipeline("question-answering", tokenizer=tokenizer, model=model)

In [69]:
questions[4997]

'가족들이랑 살다가 원룸을 구해서 나왔는데 전입신고를 꼭 해야 되는 건가요?'

In [70]:
top5_idx[4997]

[4999, 9261, 6967, 1990, 4262]

In [75]:
len(data)

9580

In [72]:
for d in data[:3]:
    print(d)

이 법은 상가건물 임대차에 관하여 「민법」에 대한 특례를 규정하여 국민 경제생활의 안정을 보장함을 목적으로 한다.

이 법은 상가건물의 임대차에 대하여 적용한다. 다만, 제14조의. 따른 상가건물임대차위원회의 심의를 거쳐 대통령령으로 정하는 보증금액을 초과하는 임대차에 대하여는 그러하지 아니하다.제. 단서에 따른 보증금액을 정할 때에는 해당 지역의 경제 여건 및 임대차 목적물의 규모 등을 고려하여 지역별로 구분하여 규정하되, 보증금 외에 차임이 있는 경우에는 그 차임액에 「은행법」에 따른 은행의 대출금리 등을 고려하여 대통령령으로 정하는 비율을 곱하여 환산한 금액을 포함하여야 한다.제. 단서에도 불구하고 제3조, 제10조제1항, 제2항, 제. 본문, 제10조의2부터 제10조의9까지의 규정 및 제19조는 제. 단서에 따른 보증금액을 초과하는 임대차에 대하여도 적용한다.

임대차는 그 등기가 없는 경우에도 임차인이 건물의 인도와 「부가가치세법」 제8조, 「소득세법」 제16. 또는 「법인세법」 제111조에 따른 사업자등록을 신청하면 그 다음 날부터 제3자에 대하여 효력이 생긴다.임차건물의 양수인은 임대인의 지위를 승계한 것으로 본다.이 법에 따라 임대차의 목적이 된 건물이 매매 또는 경매의 목적물이 된 경우에는 「민법」 제575조제1항ㆍ제. 및 제57. 를 준용한다.제3항의 경우에는 「민법」 제536조를 준용한다.



In [74]:
for i in top5_idx[4997]:
    pprint(qa({
        "question": questions[4997],
        "context": data[i],
    }))

{'answer': '3. 이상으로', 'end': 120, 'score': 0.9872922301292419, 'start': 113}
{'answer': '흘려보낼', 'end': 159, 'score': 2.006984686886426e-06, 'start': 155}
{'answer': '배당소득에', 'end': 25, 'score': 0.008753363974392414, 'start': 20}
{'answer': '지정ㆍ고시된 날부터', 'end': 76, 'score': 8.247970991089038e-13, 'start': 66}
{'answer': '"대통령령으로 정하는 범위"란',
 'end': 32,
 'score': 0.003323178505524993,
 'start': 16}
